# MDM 3 Project 2 - Interdisciplinarity

In [ ]:
"""
Author(s) - Mike Talbot
Date Created - 08/12/18
Date Modified - 08/12/18

Utility:
Code created to pull 'Authors_ID' from the supplied articles.csv file in order
to assign connections between authors and be able to compare who works with who.

Features:
Ability to pull 'Authours_ID' from File
Recreates a 2D array containing every Author for each Article
Creates a list of each separate Author. {Currently a reduced list due to the
high volume of data, not a complaint, the more the merrier but I want my laptop
to not hate me ;P }
Creates a connectivity matrix based on who has worked on what article, only
tested on small data sets so far (Controlled by the variable "Article_Limit")

Note:
Please feel free to adapt and change code, just make notes if you are changing
any purposes and wack on any credits you want ;P
"""

In [ ]:
"""
EDITS LOG:

08/12/18: Created file to read the articles.csv file and take out the authors_id 
allowing us to create a large connectivity matrix

"""

In [ ]:
"""
TODO LOG:
(More just an ideas place)

Make a network pretty
Make unefficeint code efficient!
Make journals.csv work

"""

In [1]:
"""
Cool fresh imports all the way from the internets to make the magic happen!
"""
import pandas
import numpy as np

In [2]:
"""
Practising how pandas are useful in extracting data, currently (as of 08/12/18)
errors in reading Journals.csv due to a parsing error

TOP TIPS:
x.head() shows the top 5 rows in a dataset
x.tail() shows the bottom 5 rows in a dataset
^^^^ both of these are produced in a real nice way to see the data!
x["TITLE_NAME"] extracts that columns data with their respective index
"""

#Authors = pandas.read_csv('authors.csv')
Articles = pandas.read_csv('articles.csv')
#Journals = pandas.read_csv('journals.csv')
#Authors.head()
#Authors.tail()
#Journals.head()
#Articles.head()

"""
Code to take out the authors ID from the input data and separate out each author
A check is made to remove and blanks spaces which can be contained from how the
variables were stored within their .csv files.
"""

Articles_ID = Articles["Author(s) ID"]
Articles_ID_Sep = [x.split(';')for x in Articles_ID]
for x in Articles_ID_Sep:
    if '' in x:
        x = x.remove('')
        
    
#print(Articles_ID_Sep[0:5])
#Articles_ID.head()

In [3]:
"""
Code reducing the complex array formed above into a list of each independant
author, hopefully helping to create a framework that will allow us to create
a connetivity matrix :D
A variable "Article_Limit" has been added to reduce the number of data sets the
code is running with for a faster prototyping phase

NOTE:
A warning message does appear in this section, quote:
"FutureWarning: elementwise comparison failed; returning scalar instead, but in
the future will perform elementwise comparison"
No issues have been caused so far, but just so you are warned :)
"""
Article_Limit = 10

Total_Authors = np.array([])
for Author_List in Articles_ID_Sep[0:Article_Limit]:
    for author in Author_List:
        if author not in Total_Authors:
            Total_Authors = np.append(Total_Authors,author)
#print(Total_Authors)

C:\Users\micha\Anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [4]:
"""
In theory this section will atttempt to make a NxN matrix, where N is the number
of independant authors. The values in said matrix are the number of times each
researcher has worked with each other.
This method is far from efficient and I would very much apprecciate any advice on
how to speed things up, but it is a working method. Like with three nested for
loops like damn thats a lot of looping.
"""
Number_Of_Authors = len(Total_Authors)
Connectivity = np.zeros((Number_Of_Authors,Number_Of_Authors))

for Article_Authors in Articles_ID_Sep[0:Article_Limit]:
    for Original_Author in Total_Authors:
        for Compared_Author in Total_Authors:
            if Compared_Author != Original_Author:
                if (Compared_Author in Article_Authors \
                and Original_Author in Article_Authors):
                    O = np.where(Total_Authors == Original_Author)
                    C = np.where(Total_Authors == Compared_Author)
                    Connectivity[O,C] += 1
#print(Connectivity)                        

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
